# Exercício de Análise de Dados 
*** 
<br>

### Cálculo da soma de remunerações em um dataframe a partir de uma lista importada de arquivos PDF.
<br>

<br>

### Importação dos Módulos

<br>

In [1]:
#utilizei o camelot pela sintaxe simples e eficiência
import os
import numpy as np
import pandas as pd
import camelot
import zipfile

<br>

### Importação dos Arquivos PDF e Concatenação da Listagem
<br>
Obs. O Camelot possui um bug: a seleção de várias páginas, (ex. pages='1-3' ou 'all'), não funciona. Então foi necessaŕio repetir a rotina de extração para cada página.

<br>

Isso também fez com que a rotina não funcionasse para o último arquivo, que só possui uma página, gerando erro de índice e obrigando a tratar esse último arquivo separadamente. 
<br>
<br>

In [2]:
List_Path = './Lista_Completa/Arquivos/'
Ult_Path = './Lista_Completa/ult/'
Lista_Completa = pd.DataFrame()
df3 = pd.DataFrame(columns=[0,1,2,3,4,5,6,7,8])

# Leitura dos arquivos de 3 páginas
for arq in os.listdir(List_Path):
    df1 = camelot.read_pdf(List_Path+arq, pages='1') 
    df2 = df1[0].df
    df3 = df3.append(df2)
    df1 = camelot.read_pdf(List_Path+arq, pages='2') 
    df2 = df1[0].df
    df3 = df3.append(df2)
    df1 = camelot.read_pdf(List_Path+arq, pages='3') 
    df2 = df1[0].df
    df3 = df3.append(df2)
    
#Leitura do último arquivo com uma só página
df1 = camelot.read_pdf(List_Path+arq, pages='1')
df2 = df1[0].df
df3 = df3.append(df2)

df3.rename(columns=df3.iloc[0]).drop(df3.index[0])
df3.columns = ['A', 'B', 'C', 'NOME', 'E', 'F', 'G', 'H', 'I']
df3.drop(df3[['A', 'B', 'C', 'E', 'F', 'G', 'H', 'I']], axis=1, inplace=True)
df3.drop(df3.index[0], axis=0, inplace=True)

#Criação da Lista Completa
Lista_Completa = Lista_Completa.append(df3)
Lista_Completa

,NOME
1,ABDO HASSEN SALMAN
2,ADAILTON ABREU DA SILVA
3,ADALTON FARIA
4,ADEMILDES FERREIRA DE LIMA
5,ADEMIR CARNEIRO DA CUNHA
...,...
10,WALTER FERREIRA DOS SANTOS
11,WALTER PIZZO JUNIOR
12,WALTER STEFANUTO
13,WALTNA PANTOJA DE BRITO\nCARVALHO


<br>

### Explorando o Arquivo Geral zipado

<br>

In [21]:
with zipfile.ZipFile('Servidores.zip') as BigFile:
    print(BigFile.namelist(),sep='\n')

['202001_Afastamentos.csv', '202001_Cadastro.csv', '202001_Honorarios(Jetons).csv', '202001_HonorariosAdvocaticios.csv', '202001_Observacoes.csv', '202001_Remuneracao.csv']


<br>

### Preparação do Arquivo selecionado dentro do zip como DataFrame

<br>

In [4]:
with zipfile.ZipFile('Servidores.zip') as BigFile:
    with BigFile.open('Servidores/202001_Remuneracao.csv') as data:
        dfBig = pd.read_csv(data, sep=';', decimal=",", low_memory=False, encoding='ISO-8859-1') #testar com nrows=10000 primeiro

<br>

### Filtragem dos Dados

<br>

In [5]:
dfBig.columns

Index(['ANO', 'MES', 'Id_SERVIDOR_PORTAL', 'CPF', 'NOME',
       'REMUNERAÇÃO BÁSICA BRUTA (R$)', 'REMUNERAÇÃO BÁSICA BRUTA (U$)',
       'ABATE-TETO (R$)', 'ABATE-TETO (U$)', 'GRATIFICAÇÃO NATALINA (R$)',
       'GRATIFICAÇÃO NATALINA (U$)',
       'ABATE-TETO DA GRATIFICAÇÃO NATALINA (R$)',
       'ABATE-TETO DA GRATIFICAÇÃO NATALINA (U$)', 'FÉRIAS (R$)',
       'FÉRIAS (U$)', 'OUTRAS REMUNERAÇÕES EVENTUAIS (R$)',
       'OUTRAS REMUNERAÇÕES EVENTUAIS (U$)', 'IRRF (R$)', 'IRRF (U$)',
       'PSS/RPGS (R$)', 'PSS/RPGS (U$)', 'DEMAIS DEDUÇÕES (R$)',
       'DEMAIS DEDUÇÕES (U$)', 'PENSÃO MILITAR (R$)', 'PENSÃO MILITAR (U$)',
       'FUNDO DE SAÚDE (R$)', 'FUNDO DE SAÚDE (U$)',
       'TAXA DE OCUPAÇÃO IMÓVEL FUNCIONAL (R$)',
       'TAXA DE OCUPAÇÃO IMÓVEL FUNCIONAL (U$)',
       'REMUNERAÇÃO APÓS DEDUÇÕES OBRIGATÓRIAS (R$)',
       'REMUNERAÇÃO APÓS DEDUÇÕES OBRIGATÓRIAS (U$)',
       'VERBAS INDENIZATÓRIAS REGISTRADAS EM SISTEMAS DE PESSOAL - CIVIL (R$)(*)',
       'VERBAS INDENIZATÓ

In [6]:
dfBig[['REMUNERAÇÃO BÁSICA BRUTA (R$)']] #Seleção do campo a ser processado

,REMUNERAÇÃO BÁSICA BRUTA (R$)
0,7967.90
1,34006.68
2,14249.03
3,21007.45
4,5896.78
...,...
554384,19646.82
554385,4801.59
554386,6612.02
554387,17945.81


<br>

### Eliminando registros inválidos

<br>

In [7]:
dfBig.loc[dfBig["REMUNERAÇÃO BÁSICA BRUTA (R$)"].isnull()]  #verificando valores nulos para elaborar a melhor estratégia

,ANO,MES,Id_SERVIDOR_PORTAL,CPF,NOME,REMUNERAÇÃO BÁSICA BRUTA (R$),REMUNERAÇÃO BÁSICA BRUTA (U$),ABATE-TETO (R$),ABATE-TETO (U$),GRATIFICAÇÃO NATALINA (R$),...,REMUNERAÇÃO APÓS DEDUÇÕES OBRIGATÓRIAS (R$),REMUNERAÇÃO APÓS DEDUÇÕES OBRIGATÓRIAS (U$),VERBAS INDENIZATÓRIAS REGISTRADAS EM SISTEMAS DE PESSOAL - CIVIL (R$)(*),VERBAS INDENIZATÓRIAS REGISTRADAS EM SISTEMAS DE PESSOAL - CIVIL (U$)(*),VERBAS INDENIZATÓRIAS REGISTRADAS EM SISTEMAS DE PESSOAL - MILITAR (R$)(*),VERBAS INDENIZATÓRIAS REGISTRADAS EM SISTEMAS DE PESSOAL - MILITAR (U$)(*),VERBAS INDENIZATÓRIAS PROGRAMA DESLIGAMENTO VOLUNTÁRIO  MP 792/2017 (R$),VERBAS INDENIZATÓRIAS PROGRAMA DESLIGAMENTO VOLUNTÁRIO  MP 792/2017 (U$),TOTAL DE VERBAS INDENIZATÓRIAS (R$)(*),TOTAL DE VERBAS INDENIZATÓRIAS (U$)(*)
554388,(*) Verbas indenizatórias são as parcelas inde...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
dfBig = dfBig.drop(dfBig.index[554388]) #como só há um valor nulo é melhor simplesmente excuí-lo

In [9]:
dfBig.drop_duplicates()  #Eliminando valores duplicados, se houver

,ANO,MES,Id_SERVIDOR_PORTAL,CPF,NOME,REMUNERAÇÃO BÁSICA BRUTA (R$),REMUNERAÇÃO BÁSICA BRUTA (U$),ABATE-TETO (R$),ABATE-TETO (U$),GRATIFICAÇÃO NATALINA (R$),...,REMUNERAÇÃO APÓS DEDUÇÕES OBRIGATÓRIAS (R$),REMUNERAÇÃO APÓS DEDUÇÕES OBRIGATÓRIAS (U$),VERBAS INDENIZATÓRIAS REGISTRADAS EM SISTEMAS DE PESSOAL - CIVIL (R$)(*),VERBAS INDENIZATÓRIAS REGISTRADAS EM SISTEMAS DE PESSOAL - CIVIL (U$)(*),VERBAS INDENIZATÓRIAS REGISTRADAS EM SISTEMAS DE PESSOAL - MILITAR (R$)(*),VERBAS INDENIZATÓRIAS REGISTRADAS EM SISTEMAS DE PESSOAL - MILITAR (U$)(*),VERBAS INDENIZATÓRIAS PROGRAMA DESLIGAMENTO VOLUNTÁRIO  MP 792/2017 (R$),VERBAS INDENIZATÓRIAS PROGRAMA DESLIGAMENTO VOLUNTÁRIO  MP 792/2017 (U$),TOTAL DE VERBAS INDENIZATÓRIAS (R$)(*),TOTAL DE VERBAS INDENIZATÓRIAS (U$)(*)
0,2020,1.0,5116961.0,***.017.623-**,AARAO CARLOS LUZ MACAMBIRA,7967.90,0.0,0.0,0.0,0.0,...,8994.22,0.0,779.00,0.0,0.0,0.0,0.0,0.0,779.00,0.0
1,2020,1.0,201964.0,***.056.281-**,AARAO DIAMANTINO OLIVEIRA,34006.68,0.0,0.0,0.0,0.0,...,25794.21,0.0,458.00,0.0,0.0,0.0,0.0,0.0,458.00,0.0
2,2020,1.0,4405000.0,***.116.132-**,AARAO FERREIRA LIMA NETO,14249.03,0.0,0.0,0.0,0.0,...,10141.35,0.0,458.00,0.0,0.0,0.0,0.0,0.0,458.00,0.0
3,2020,1.0,4915841.0,***.031.184-**,AARAO PEREIRA DE ARAUJO JUNIOR,21007.45,0.0,0.0,0.0,0.0,...,14528.70,0.0,798.12,0.0,0.0,0.0,0.0,0.0,798.12,0.0
4,2020,1.0,5105492.0,***.859.807-**,AARAO SOARES,5896.78,0.0,0.0,0.0,0.0,...,4555.22,0.0,1031.32,0.0,0.0,0.0,0.0,0.0,1031.32,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
554383,2020,1.0,7400917.0,***.425.967-**,ZURLINDE CABRAL LOURO JUNIOR,11643.75,0.0,0.0,0.0,0.0,...,9208.71,0.0,595.53,0.0,0.0,0.0,0.0,0.0,595.53,0.0
554384,2020,1.0,216805.0,***.651.506-**,ZUY MARIA MAGRIOTIS,19646.82,0.0,0.0,0.0,0.0,...,13624.88,0.0,579.14,0.0,0.0,0.0,0.0,0.0,579.14,0.0
554385,2020,1.0,1307290.0,***.034.989-**,ZUZANA RENDAKI MIKOS,4801.59,0.0,0.0,0.0,0.0,...,4300.70,0.0,732.18,0.0,0.0,0.0,0.0,0.0,732.18,0.0
554386,2020,1.0,8504240.0,***.041.463-**,ZUZITA DOS SANTOS VIANA,6612.02,0.0,0.0,0.0,0.0,...,5663.08,0.0,588.71,0.0,0.0,0.0,0.0,0.0,588.71,0.0


In [10]:
dfBig.loc[dfBig["REMUNERAÇÃO BÁSICA BRUTA (R$)"] == 0] # verificando registros com remuneração = zero

,ANO,MES,Id_SERVIDOR_PORTAL,CPF,NOME,REMUNERAÇÃO BÁSICA BRUTA (R$),REMUNERAÇÃO BÁSICA BRUTA (U$),ABATE-TETO (R$),ABATE-TETO (U$),GRATIFICAÇÃO NATALINA (R$),...,REMUNERAÇÃO APÓS DEDUÇÕES OBRIGATÓRIAS (R$),REMUNERAÇÃO APÓS DEDUÇÕES OBRIGATÓRIAS (U$),VERBAS INDENIZATÓRIAS REGISTRADAS EM SISTEMAS DE PESSOAL - CIVIL (R$)(*),VERBAS INDENIZATÓRIAS REGISTRADAS EM SISTEMAS DE PESSOAL - CIVIL (U$)(*),VERBAS INDENIZATÓRIAS REGISTRADAS EM SISTEMAS DE PESSOAL - MILITAR (R$)(*),VERBAS INDENIZATÓRIAS REGISTRADAS EM SISTEMAS DE PESSOAL - MILITAR (U$)(*),VERBAS INDENIZATÓRIAS PROGRAMA DESLIGAMENTO VOLUNTÁRIO  MP 792/2017 (R$),VERBAS INDENIZATÓRIAS PROGRAMA DESLIGAMENTO VOLUNTÁRIO  MP 792/2017 (U$),TOTAL DE VERBAS INDENIZATÓRIAS (R$)(*),TOTAL DE VERBAS INDENIZATÓRIAS (U$)(*)
33,2020,1.0,113816702.0,***.828.223-**,ABDALA DA COSTA SOUSA,0.0,0.0,0.0,0.0,0.0,...,11715.39,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0
35,2020,1.0,9116248.0,***.672.871-**,ABDALAH ALI ABDEL CADER,0.0,0.0,0.0,0.0,0.0,...,11715.39,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0
46,2020,1.0,123316827.0,***.454.252-**,ABDENEGO NERI CORREIA,0.0,0.0,0.0,0.0,0.0,...,11715.39,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0
51,2020,1.0,7104539.0,***.008.102-**,ABDIAS DA SILVA MATOS,0.0,0.0,0.0,0.0,0.0,...,2348.84,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0
109,2020,1.0,101918447.0,***.344.892-**,ABEGNO GADELHA DE MENDONCA,0.0,0.0,0.0,0.0,0.0,...,11715.39,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
554153,2020,1.0,61802705.0,***.125.841-**,ZOILA REYES HERRERA CORREIA DA COSTA,0.0,0.0,0.0,0.0,0.0,...,11715.39,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0
554268,2020,1.0,82441284.0,***.739.398-**,ZULEIDE ALVES GONCALVES,0.0,0.0,0.0,0.0,0.0,...,11715.39,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0
554304,2020,1.0,31989788.0,***.890.443-**,ZULEIDE OLIVEIRA FEITOSA,0.0,0.0,0.0,0.0,33.4,...,1068.03,0.0,4373.41,0.0,0.0,0.0,0.0,0.0,4373.41,0.0
554338,2020,1.0,8802376.0,***.960.641-**,ZULLYBEL ANDREINA DUBEN FLORES,0.0,0.0,0.0,0.0,0.0,...,11715.39,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0


<br>

## Substituir valores "zero" pela média.

<br>

Obs. Não há o campo "Órgão"na lista geral, mas como todos da lista de origem são do Ministério da Economia, com uma média salarial muito mais alta que a geral (~20.000 ao invés de ~8.000). Talvez fosse mais preciso substituir pela média da própria lista.

<br>

In [11]:
SalMedia = dfBig["REMUNERAÇÃO BÁSICA BRUTA (R$)"].median()
SalMedia

8232.64

In [12]:
dfBig['REMUNERAÇÃO BÁSICA BRUTA (R$)'] = dfBig['REMUNERAÇÃO BÁSICA BRUTA (R$)'].replace(0,SalMedia) #Substituindo valores zero pela média

In [13]:
dfWork = pd.DataFrame(dfBig, columns= ['NOME','REMUNERAÇÃO BÁSICA BRUTA (R$)'])
dfWork

,NOME,REMUNERAÇÃO BÁSICA BRUTA (R$)
0,AARAO CARLOS LUZ MACAMBIRA,7967.90
1,AARAO DIAMANTINO OLIVEIRA,34006.68
2,AARAO FERREIRA LIMA NETO,14249.03
3,AARAO PEREIRA DE ARAUJO JUNIOR,21007.45
4,AARAO SOARES,5896.78
...,...,...
554383,ZURLINDE CABRAL LOURO JUNIOR,11643.75
554384,ZUY MARIA MAGRIOTIS,19646.82
554385,ZUZANA RENDAKI MIKOS,4801.59
554386,ZUZITA DOS SANTOS VIANA,6612.02


In [14]:
dfWork['REMUNERAÇÃO BÁSICA BRUTA (R$)'].astype(np.float32)

0          7967.899902
1         34006.679688
2         14249.030273
3         21007.449219
4          5896.779785
              ...     
554383    11643.750000
554384    19646.820312
554385     4801.589844
554386     6612.020020
554387    17945.810547
Name: REMUNERAÇÃO BÁSICA BRUTA (R$), Length: 554388, dtype: float32

In [15]:
dfFinal = dfWork.loc[dfBig['NOME'].isin(Lista_Completa['NOME'])]
dfFinal

,NOME,REMUNERAÇÃO BÁSICA BRUTA (R$)
75,ABDO HASSEN SALMAN,19280.76
830,ADAILTON ABREU DA SILVA,17579.95
1320,ADALTON FARIA,20468.03
2484,ADEMILDES FERREIRA DE LIMA,17952.92
2591,ADEMIR CARNEIRO DA CUNHA,21099.50
...,...,...
551819,WLADIMIR DE SOUSA E SILVA,15824.37
551880,WLADMIR MARTINS JUNIOR,16489.15
551949,WOLMIR MELLO TEIXEIRA DE FREITAS,20468.03
551981,WOODROW DELANO WILSON,20468.03


<br>

### Soma Final da Remuneração dos Servidores da Lista

<br>

In [25]:
SOMA = dfFinal['REMUNERAÇÃO BÁSICA BRUTA (R$)'].sum()
print("A soma de todas as reminerações da lista é: ")
print(SOMA)

A soma de todas as reminerações da lista é: 
63563847.37
